# Probabilités & Statistiques  
  
### Septembre 2022
<br/>

## Série 01

---

In [1]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)
PDF('res/serie01.pdf', size=(800, 400))

### 1. Dénombrabilité
---

On cherche à écrire un programme qui donné un `n` nous donne le n-ème élément de $A^{*} = {\{0,1\}^{*}}$

A* est ordonné selon l'ordre lexicographique, i.e. $A^*=\{0; 1; 00; 01; 10; 11; 000; 001; 010 : 011; 100; 101; 110; 111;. . .\}$  
<br/>

Définisson ${B=A^{*}}\ $ par simplicité. Si on sépare $B$ en "groupe" d'éléments de même tailles (i.e. les 0,1, les 00, 01 ...),  
on se rend compte que le nombre d'élément de chaque groupe est donné par $2^k$ où $k$ est la taille des éléments de ce dernier.  
On a donc que le nombre d'éléments (possibilités de combinaison de 0,1) de taille $\leq k$ est
$\sum^{k}_{i=1}{2^i} = 2^{i+1}-2$  
<br/>

Avec notre `n` donné, on cherche quelle taille ($k$) correspond à au moins `n` éléments:  
$2^{k+1}-2 = n \Longleftrightarrow 2^{k} = \frac{n}{2}+1 \Longleftrightarrow k = \log_2{( \frac{n}{2} + 1 )}$  
On obtient donc la taille du n-ème élément de $B$ comme $|B_n| = \lceil{\log_2{( \frac{n}{2} + 1 )}}\rceil$

In [66]:
import math
def log2(x): 
    return math.log(x, 2)
def lg(n):
    """return length of n-th element of B"""
    return math.ceil(log2(n/2 + 1))
def sum_group_card(n):
    return 2**(n+1) - 2

In [65]:
def getIndexInGroup(n):
    """Return the index of B_n among all elements of same length i.e. 'it is the k-th element of length lg(n)''"""
    length = lg(n)
    indexFromGroupStart = n - sum_group_card(length-1)
    return indexFromGroupStart
# 
# Tests
n = 8
print(lg(n))
print("index in group:", getIndexInGroup(13))

def padd(s, newLen):
    """Padd string with '0' until given length"""
    crtLen = len(s)
    if crtLen >= newLen: return s
    paddNb = newLen - crtLen
    return paddNb*"0" + s

3
index in group: 7


Maintenant qu'on a la position de $B_n$ parmis les éléments de la même taille que ce dernier, il suffit juste de convertir ce nombre en binaire et rajouter des 0 (padd) devant jusqu'à atteindre la taille désirée.

In [83]:
def toBinPadd(n, newLen):
    """Convert to binary removes the '0b' prefix and padd with 0s"""
    return padd(str(bin(n))[2:], newLen)